## IMPORTS and UTILS

In [22]:
print("hi")

import os
import pandas as pd
import subprocess
import re
import numpy as np
import json
import csv

from joblib import Parallel, delayed



import matplotlib.pyplot as plt


import os

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC
from robust_evaluation_tools.robust_utils import get_site, robust_text, rwp_text, get_camcan_file, get_diseases, get_metrics, add_nb_patients_and_diseased
from robust_evaluation_tools.robust_harmonization import fit, apply, visualize_harmonization, QC, compare_with_compilation, create_presentation, compare_distances, compare_with_compilation_var
from robust_evaluation_tools.synthectic_sites_generations import generate_sites

MAINFOLDER = "RESULTS/MAE_TEST"
SYNTHETIC_SITES = f"{MAINFOLDER}/SYNTHETIC_SITES"

ANALYSIS_FOLDER = f"{MAINFOLDER}/ANALYSIS"

robust_methods_for_analysis = ["MMS","IQR",'MAD', 'VS', 'VS2', 'TOP30']

hi


## HARMONIZATION

In [ ]:
def harmonize(f_train, ref_data_file, metric,harmonizartion_method, f_test, directory, robust, rwp,hc):
    os.makedirs(directory, exist_ok=True)
    print(f_train)
    
    # Fit the model
    output_model_filename = fit(f_train, ref_data_file, metric, harmonizartion_method, robust, rwp, directory, hc,)
    # Apply the model
    output_filename = apply(f_test, output_model_filename, metric, harmonizartion_method, robust, rwp, directory)
    
    # Visualize the harmonization
    #visualize_harmonization(f_test, output_filename, ref_data_file, directory, bundles = '')
    mae = compare_with_compilation(pd.read_csv(output_filename))
    maev = compare_with_compilation_var(pd.read_csv(output_filename))

    mae['site'] = get_site(f_train)
    maev['site'] = get_site(f_train)
    
    return mae, maev

In [ ]:
def analyse_site(f_train,f_test, robust, directory, ref_data_file, metric,harmonizartion_method):
    # 4 harmonization
    harmonization_hc, maev_hc = harmonize(f_train, ref_data_file, metric,harmonizartion_method, f_test, os.path.join(directory, "hc"), "No", False, True)
    harmonization_no_robust, maev_no_robust = harmonize(f_train, ref_data_file, metric,harmonizartion_method, f_test, os.path.join(directory, "NoRobust"), "No", False, False)
    harmonization_robust, maev_robust = harmonize(f_train, ref_data_file, metric,harmonizartion_method, f_test, os.path.join(directory, "robust"), robust, False, False)
    harmonization_robust_rwp, maev_robust_rwp = harmonize(f_train, ref_data_file, metric,harmonizartion_method, f_test, os.path.join(directory, "robust_rwp"), robust, True, False)


    #create_presentation(directory, harmonizartion_method)

    # Combine MEA in a single DataFrame
    mea_combined = pd.concat([harmonization_hc, harmonization_no_robust, harmonization_robust, harmonization_robust_rwp], ignore_index=True)
    mea_combined['method'] = ['hc', 'no_robust', 'robust', 'robust_rwp']

    maev_combined = pd.concat([maev_hc, maev_no_robust, maev_robust, maev_robust_rwp], ignore_index=True)
    maev_combined['method'] = ['hc', 'no_robust', 'robust', 'robust_rwp']


    #TODO bundles et analyze outliers
    return mea_combined, maev_combined

In [ ]:
    
# #Analyse Method
# def analyse_method(sample_sizes, disease_ratios, num_tests, robust_method, SYNTHETIC_SITES_VERSION):
#     # Split the data into training and testing sets
#     directory = os.path.join(MAINFOLDER, robust_method)
#     directory_site = os.path.join(SYNTHETIC_SITES ,SYNTHETIC_SITES_VERSION)
#     # Initialize DataFrames to store the results
#     detection_measures_compilation = pd.DataFrame()
#     dists_compilation = pd.DataFrame()
#     mea_compilation = pd.DataFrame()
#     outliers_compilation = pd.DataFrame()
#     for sample_size in sample_sizes:
#         for disease_ratio in disease_ratios:        
#             sizeDir = os.path.join(directory, f"{sample_size}_{int(disease_ratio*100)}")
#             sizeDir_site = os.path.join(directory_site, f"{sample_size}_{int(disease_ratio*100)}")
#             for i in range(num_tests):
#                 tempDir = os.path.join(sizeDir, f"{i}")
#                 tempDir_site = os.path.join(sizeDir_site, f"{i}")
#                 os.makedirs(tempDir, exist_ok=True)

#                 train_file_name = f"train_{sample_size}_{int(disease_ratio*100)}_{i}.csv"
#                 test_file_name = f"test_{sample_size}_{int(disease_ratio*100)}_{i}.csv"
                
#                 # Sauvegarder l'échantillon dans un fichier temporaire
#                 train_file = os.path.join(tempDir_site,train_file_name )
#                 train_df = pd.read_csv(train_file)
#                 train_df.to_csv(os.path.join(tempDir,train_file_name ), index=False)

#                 test_file = os.path.join(tempDir_site, test_file_name)
#                 test_df = pd.read_csv(test_file)
#                 test_df.to_csv(os.path.join(tempDir,test_file_name ), index=False)

#                 ref_data_file = get_camcan_file(metric='fd')
                
#                 # Analyser le site pour le nouvel échantillon
#                 dists_analyze, mea_analyze, detection_measures, outliers = analyse_site(train_file, test_file, robust_method, tempDir, ref_data_file, metric,harmonizartion_method)
#                 analyse_site(f_train,f_test, robust, directory, ref_data_file, metric,harmonizartion_method)
#                 detection_measures_compilation = pd.concat([detection_measures_compilation, detection_measures])
#                 dists_compilation = pd.concat([dists_compilation, dists_analyze])
#                 mea_compilation = pd.concat([mea_compilation, mea_analyze])
#                 outliers_compilation = pd.concat([outliers_compilation, outliers])
#     # Save the metrics and distances compilation DataFrames to CSV files
#     detection_measures_compilation.to_csv(os.path.join(directory, "detection_measures__compilation.csv"), index=False)
#     dists_compilation.to_csv(os.path.join(directory, "dists_compilation.csv"), index=False)
#     mea_compilation.to_csv(os.path.join(directory, "mea_compilation.csv"), index=False)
#     outliers_compilation.to_csv(os.path.join(directory, "outliers_compilation.csv"), index=False)

In [ ]:

# Function to analyze a single (sample_size, disease_ratio) combination
def process_analysis(disease, metric, sample_size, disease_ratio,harmonization_method, robust_method, SYNTHETIC_SITES_VERSION, num_tests):
    directory = os.path.join(MAINFOLDER, robust_method, disease, metric)
    directory_site = os.path.join(SYNTHETIC_SITES, SYNTHETIC_SITES_VERSION,disease)
    
    sizeDir = os.path.join(directory, f"{sample_size}_{int(disease_ratio * 100)}")
    sizeDir_site = os.path.join(directory_site, f"{sample_size}_{int(disease_ratio * 100)}")

    mea_compilation = pd.DataFrame()
    maev_compilation = pd.DataFrame()

    for test_index in range(num_tests):
        tempDir = os.path.join(sizeDir, f"{test_index}")
        tempDir_site = os.path.join(sizeDir_site, f"{test_index}")
        os.makedirs(tempDir, exist_ok=True)

        train_file_name = f"train_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv"
        test_file_name = f"test_{sample_size}_{int(disease_ratio * 100)}_{test_index}_{metric}.csv"

        # Load and save training dataset
        train_file = os.path.join(tempDir_site, train_file_name)
        train_df = pd.read_csv(train_file)
        train_df = train_df[~train_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]
        train_df = train_df.drop(columns=['mean_no_cov', 'metric_bundle'])
        new_train_file = os.path.join(tempDir, train_file_name)
        train_df.to_csv(new_train_file, index=False)
        

        # Load and save test dataset
        test_file = os.path.join(tempDir_site, test_file_name)
        test_df = pd.read_csv(test_file)
        test_df = test_df[~test_df['bundle'].isin(['left_ventricle', 'right_ventricle'])]
        test_df = test_df.drop(columns=['mean_no_cov', 'metric_bundle'])
        new_test_file = os.path.join(tempDir, test_file_name)
        test_df.to_csv(new_test_file, index=False)

        ref_data_file = get_camcan_file(metric)

        # Analyze the site
        mea_analyze, maev_analyze = analyse_site(
            new_train_file, new_test_file, robust_method, tempDir, ref_data_file, metric, harmonization_method
        )
        mea_analyze['robust_method'] = robust_method
        mea_analyze['disease'] = disease
        mea_analyze['metric'] = metric

        maev_analyze['robust_method'] = robust_method
        maev_analyze['disease'] = disease
        maev_analyze['metric'] = metric

        mea_compilation = pd.concat([mea_compilation, mea_analyze], ignore_index=True)
        maev_compilation = pd.concat([maev_compilation, maev_analyze], ignore_index=True)
        

    # Sauvegarde les deux compilations
    os.makedirs(directory, exist_ok=True)
    mea_file_path = os.path.join(sizeDir, "mea_compilation.csv")
    maev_file_path = os.path.join(sizeDir, "maev_compilation.csv")

    mea_compilation.to_csv(mea_file_path, index=False)
    maev_compilation.to_csv(maev_file_path, index=False)

    # On retourne les deux chemins de fichiers
    return (mea_file_path, maev_file_path)

# Parallelized analysis method (excluding num_tests from parallelization)
def analyse_method(sample_sizes, disease_ratios, num_tests, robust_methods,diseases, metrics, harmonization_method, SYNTHETIC_SITES_VERSION):
    # Generate all task combinations (excluding num_tests)
    tasks = [
        (disease,metric, sample_size, disease_ratio,harmonization_method, robust_method, SYNTHETIC_SITES_VERSION, num_tests)
        for robust_method in robust_methods
        for disease in diseases
        for sample_size in sample_sizes
        for disease_ratio in disease_ratios
        for metric in metrics
    ]

    # Run all combinations in parallel and collect file paths
    results = Parallel(n_jobs=-1)(delayed(process_analysis)(*task) for task in tasks)

     # On sépare la liste de tuples en deux listes de chemins
    mea_file_paths = [res[0] for res in results]
    maev_file_paths = [res[1] for res in results]

    # Concatène toutes les compilations mea
    mea_compilation_global = pd.concat(
        [pd.read_csv(fpath) for fpath in mea_file_paths],
        ignore_index=True
    )

    # Concatène toutes les compilations maev
    maev_compilation_global = pd.concat(
        [pd.read_csv(fpath) for fpath in maev_file_paths],
        ignore_index=True
    )

    # Sauvegarde finale des deux DataFrames compilés
    directory = os.path.join(MAINFOLDER)
    os.makedirs(directory, exist_ok=True)

    mea_compilation_global.to_csv(os.path.join(directory, "mea_compilation_global.csv"), index=False)
    maev_compilation_global.to_csv(os.path.join(directory, "maev_compilation_global.csv"), index=False)


In [ ]:
def generate_sites_for_disease(disease, SYNTHETIC_SITES, SYNTHETIC_SITES_VERSION, sample_sizes, disease_ratios, num_tests):
    # Load data for the disease
    data_path = path = os.path.join('DONNES','COMPILATIONS_AUG_3', f'{disease}_combination_all_metrics_CamCAN.csv.gz')

    # Define site directory
    directory_site = os.path.join(SYNTHETIC_SITES, SYNTHETIC_SITES_VERSION, disease)

    # Generate synthetic sites
    generate_sites(sample_sizes, disease_ratios, num_tests, directory_site, data_path, disease=None)

## EXECUTOR

In [ ]:
harmonization_method= "classic"

SYNTHETIC_SITES_VERSION = "v1"

metrics = get_metrics()
#diseases = get_diseases(True)
diseases = ["AD","SYN_2"]
robust_methods = ["MMS","IQR",'MAD', 'VS', 'VS2', 'TOP30', 'TOP50']


sample_sizes = [100]  # Différentes tailles d'échantillon
disease_ratios = [0.1, 0.3, 0.5]  # Différents pourcentages de malades
num_tests = 6  # Nombre de tests à effectuer pour chaque combinaison

# Parallel(n_jobs=-1)(
#     delayed(generate_sites_for_disease)(disease, SYNTHETIC_SITES, SYNTHETIC_SITES_VERSION, sample_sizes, disease_ratios, num_tests)
#     for disease in diseases
# )

In [ ]:
analyse_method(sample_sizes, disease_ratios, num_tests, robust_methods,diseases, metrics, harmonization_method, SYNTHETIC_SITES_VERSION)

In [ ]:
# Charger le fichier principal
mea_compilation = pd.read_csv(os.path.join(MAINFOLDER, "mea_compilation_global.csv"))

# Récupérer les méthodes uniques
methods = mea_compilation['robust_method'].unique()

# Sauvegarder un fichier CSV pour chaque méthode
for method in methods:
    # Filtrer les lignes correspondant à la méthode
    method_df = mea_compilation[mea_compilation['robust_method'] == method]
    
    # Créer un dossier pour la méthode
    method_directory = os.path.join(MAINFOLDER, method)
    os.makedirs(method_directory, exist_ok=True)
    
    # Sauvegarder le fichier CSV dans le dossier
    method_file_path = os.path.join(method_directory, f"{method}_mea_compilation.csv")
    method_df.to_csv(method_file_path, index=False)

In [ ]:
# Charger le fichier principal
maev_compilation = pd.read_csv(os.path.join(MAINFOLDER, "maev_compilation_global.csv"))

# Récupérer les méthodes uniques
methods = maev_compilation['robust_method'].unique()

# Sauvegarder un fichier CSV pour chaque méthode
for method in methods:
    # Filtrer les lignes correspondant à la méthode
    method_df = maev_compilation[maev_compilation['robust_method'] == method]
    
    # Créer un dossier pour la méthode
    method_directory = os.path.join(MAINFOLDER, method)
    os.makedirs(method_directory, exist_ok=True)
    
    # Sauvegarder le fichier CSV dans le dossier
    method_file_path = os.path.join(method_directory, f"{method}_maev_compilation.csv")
    method_df.to_csv(method_file_path, index=False)

In [ ]:
def gather_all_method_files(main_folder, methods):
    all_dfs = []
    for m in methods:
        path = os.path.join(main_folder, m, f"{m}_mea_compilation.csv")
        if os.path.exists(path):
            df = pd.read_csv(path)
            all_dfs.append(df)
    return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()

In [ ]:
def gather_all_method_files_maev(main_folder, methods):
    all_dfs = []
    for m in methods:
        path = os.path.join(main_folder, m, f"{m}_maev_compilation.csv")
        if os.path.exists(path):
            df = pd.read_csv(path)
            all_dfs.append(df)
    return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()

## ANALYSIS

In [23]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

def plot_mea(df, sample_size, disease, metric, directory):
    directory = os.path.join(directory, "MAE_PLOTS", disease, metric, str(sample_size))
    os.makedirs(directory, exist_ok=True)
    
    df_filtered = df[
        (df['num_patients'] == sample_size) & 
        (df['disease'] == disease) & 
        (df['metric'] == metric)
    ]

    # Méthodes de base + méthodes robustes
    robust_methods = df_filtered['robust_method'].dropna().unique()
    base_methods = ["hc", "no_robust"]
    robust_base_methods = ["robust"]
    methods = base_methods + [f"{rb}_{rm}" for rb in robust_base_methods for rm in robust_methods]
    
    # Couleurs
    method_colors = {
        "hc": "green",
        "no_robust": "red"
    }
    robust_palette = sns.color_palette("viridis", len(robust_methods))
    robust_rwp_palette = sns.color_palette("magma", len(robust_methods))
    for i, rm in enumerate(robust_methods):
        method_colors[f"robust_{rm}"] = robust_palette[i]
        method_colors[f"robust_rwp_{rm}"] = robust_rwp_palette[i]

    # Boucle sur les "bundles"
    for bundle_column in df_filtered.columns:
        # On ignore les colonnes non-numériques
        if bundle_column in ['site', 'method', 'num_patients', 'disease_ratio',
                             'num_diseased', 'metric', 'disease', 'robust_method']:
            continue

        bundle_df = df_filtered[[bundle_column, 'site', 'method', 'num_patients',
                                 'disease_ratio', 'num_diseased', 'robust_method']].copy()
        unique_ratios = sorted(bundle_df['disease_ratio'].unique())  # Trié pour être sûr de l'ordre

        # *** ICI on paramètre la figure + le positionnement ***
        fig, ax = plt.subplots(figsize=(14, 7))  # Ajuster au besoin

        # Abscisses pour chaque ratio
        x = np.arange(len(unique_ratios))
        
        # Largeur totale allouée pour le « groupe » de méthodes à chaque ratio
        group_width = 0.8  
        # On répartit cette largeur entre toutes les méthodes
        n_methods = len(methods)
        box_width = group_width / n_methods

        # Tracé des boxplots pour chaque méthode
        for i_m, method in enumerate(methods):
            if "_" in method and method != "no_robust":
                method_base, robust_type = method.rsplit("_", 1)
                method_df = bundle_df[
                    (bundle_df['method'] == method_base) & 
                    (bundle_df['robust_method'] == robust_type)
                ]
            else:
                method_df = bundle_df[bundle_df['method'] == method]

            # On prépare la liste de valeurs par ratio
            data = [
                method_df[method_df['disease_ratio'] == ratio][bundle_column].values 
                for ratio in unique_ratios
            ]

            # Positions: on centre autour de chaque x
            # Exemple: x - group_width/2 + (i_m+0.5)*box_width
            positions = x - group_width/2 + (i_m + 0.5)*box_width

            color = method_colors.get(method, "black")
            
            # S’il y a au moins un point de données
            if any(len(d) > 0 for d in data):
                ax.boxplot(
                    data,
                    positions=positions,
                    widths=box_width * 0.8,  # Légèrement plus petit que box_width
                    patch_artist=True,
                    boxprops=dict(facecolor=color, color=color),
                    medianprops=dict(color='black')
                )

        ax.set_xlabel('Prct de patients malades')
        ax.set_ylabel('MAE')
        ax.set_title(
            f"MAE de l'harmonization selon le pourcentage de patients malades\n"
            f"Maladie: {disease}  |  Metric: {metric}  |  Bundle: {bundle_column}\n"
            f"Nb patient total: {sample_size}"
        )

        # On place les ticks au milieu de chaque groupe (i.e. sur x)
        ax.set_xticks(x)
        ax.set_xticklabels(unique_ratios)

        # Légende manuelle
        legend_handles = [
            plt.Line2D([0], [0], color=method_colors[m], lw=3, label=f'Method: {m}')
            for m in methods
        ]
        ax.legend(handles=legend_handles, loc="upper left", bbox_to_anchor=(1, 1))

        plt.tight_layout()
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'), bbox_inches="tight")
        plt.close()


# Load dataset
mea_df = gather_all_method_files(MAINFOLDER, robust_methods_for_analysis)
add_nb_patients_and_diseased(mea_df)

# Generate all task combinations
tasks = [
    (mea_df, sample_size, disease, metric, ANALYSIS_FOLDER)
    for disease in diseases
    for sample_size in sample_sizes
    for metric in metrics
]

# Run all tasks in parallel
Parallel(n_jobs=-1)(
    delayed(plot_mea)(*task) for task in tasks
)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed

def plot_maev(df, sample_size, disease, metric, directory):
    directory = os.path.join(directory, "MAEV_PLOTS", disease, metric, str(sample_size))
    os.makedirs(directory, exist_ok=True)
    
    df_filtered = df[
        (df['num_patients'] == sample_size) & 
        (df['disease'] == disease) & 
        (df['metric'] == metric)
    ]

    # Méthodes de base + méthodes robustes
    robust_methods = df_filtered['robust_method'].dropna().unique()
    base_methods = ["hc", "no_robust"]
    robust_base_methods = ["robust",]
    methods = base_methods + [f"{rb}_{rm}" for rb in robust_base_methods for rm in robust_methods]
    
    # Couleurs
    method_colors = {
        "hc": "green",
        "no_robust": "red"
    }
    robust_palette = sns.color_palette("viridis", len(robust_methods))
    robust_rwp_palette = sns.color_palette("magma", len(robust_methods))
    for i, rm in enumerate(robust_methods):
        method_colors[f"robust_{rm}"] = robust_palette[i]
        method_colors[f"robust_rwp_{rm}"] = robust_rwp_palette[i]

    # Boucle sur les "bundles"
    for bundle_column in df_filtered.columns:
        # On ignore les colonnes non-numériques
        if bundle_column in ['site', 'method', 'num_patients', 'disease_ratio',
                             'num_diseased', 'metric', 'disease', 'robust_method']:
            continue

        bundle_df = df_filtered[[bundle_column, 'site', 'method', 'num_patients',
                                 'disease_ratio', 'num_diseased', 'robust_method']].copy()
        unique_ratios = sorted(bundle_df['disease_ratio'].unique())  # Trié pour être sûr de l'ordre

        # *** ICI on paramètre la figure + le positionnement ***
        fig, ax = plt.subplots(figsize=(14, 7))  # Ajuster au besoin

        # Abscisses pour chaque ratio
        x = np.arange(len(unique_ratios))
        
        # Largeur totale allouée pour le « groupe » de méthodes à chaque ratio
        group_width = 0.8  
        # On répartit cette largeur entre toutes les méthodes
        n_methods = len(methods)
        box_width = group_width / n_methods

        # Tracé des boxplots pour chaque méthode
        for i_m, method in enumerate(methods):
            if "_" in method and method != "no_robust":
                method_base, robust_type = method.rsplit("_", 1)
                method_df = bundle_df[
                    (bundle_df['method'] == method_base) & 
                    (bundle_df['robust_method'] == robust_type)
                ]
            else:
                method_df = bundle_df[bundle_df['method'] == method]

            # On prépare la liste de valeurs par ratio
            data = [
                method_df[method_df['disease_ratio'] == ratio][bundle_column].values 
                for ratio in unique_ratios
            ]

            # Positions: on centre autour de chaque x
            # Exemple: x - group_width/2 + (i_m+0.5)*box_width
            positions = x - group_width/2 + (i_m + 0.5)*box_width

            color = method_colors.get(method, "black")
            
            # S’il y a au moins un point de données
            if any(len(d) > 0 for d in data):
                ax.boxplot(
                    data,
                    positions=positions,
                    widths=box_width * 0.8,  # Légèrement plus petit que box_width
                    patch_artist=True,
                    boxprops=dict(facecolor=color, color=color),
                    medianprops=dict(color='black')
                )

        ax.set_xlabel('Prct de patients malades')
        ax.set_ylabel('MAEV')
        ax.set_title(
            f"MAEV de l'harmonization selon le pourcentage de patients malades\n"
            f"Maladie: {disease}  |  Metric: {metric}  |  Bundle: {bundle_column}\n"
            f"Nb patient total: {sample_size}"
        )

        # On place les ticks au milieu de chaque groupe (i.e. sur x)
        ax.set_xticks(x)
        ax.set_xticklabels(unique_ratios)

        # Légende manuelle
        legend_handles = [
            plt.Line2D([0], [0], color=method_colors[m], lw=3, label=f'Method: {m}')
            for m in methods
        ]
        ax.legend(handles=legend_handles, loc="upper left", bbox_to_anchor=(1, 1))

        plt.tight_layout()
        plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'), bbox_inches="tight")
        plt.close()


# Load dataset
mea_df = gather_all_method_files_maev(MAINFOLDER, robust_methods_for_analysis)
add_nb_patients_and_diseased(mea_df)

# Generate all task combinations
tasks = [
    (mea_df, sample_size, disease, metric, ANALYSIS_FOLDER)
    for disease in diseases
    for sample_size in sample_sizes
    for metric in metrics
]

# Run all tasks in parallel
Parallel(n_jobs=-1)(
    delayed(plot_maev)(*task) for task in tasks
)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [25]:
# # CREATION BOX PLOT POUR MEA
# def plot_bundle(df, prct, directory):
#     """
#     Crée un graphique pour chaque bundle dans le DataFrame donné.
#     L'axe des X représente le nombre de patients et l'axe des Y représente la moyenne de la colonne du bundle.
#     La courbe inclut une zone indiquant l'écart-type (std).

#     Parameters:
#     df (pd.DataFrame): Le DataFrame contenant les données.
#     bundle_column (str): Le nom de la colonne du bundle à utiliser pour le graphique.
#     """
#     directory = os.path.join(directory, "MEA_PLOTS", str(prct))
#     df = df[df['disease_ratio'] == prct *100]
#     os.makedirs(directory, exist_ok=True)
    
#     for bundle_column in df.columns:
#         if bundle_column in ['site','method','num_patients','disease_ratio','num_diseased']:
#             continue # Skip non-numeric columns
#         bundle_df = df[[bundle_column, 'site', 'method','num_patients','disease_ratio','num_diseased']].copy()
#         methods = ["hc", "no_robust", "robust", "robust_rwp"]
#         colors = ['blue', 'green', 'red', 'purple']
        
#         plt.figure(figsize=(10, 5))
#         width = 0.2  # the width of the bars
#         x = np.arange(len(bundle_df['num_patients'].unique()))  # the label locations
#         fig, ax = plt.subplots(figsize=(10, 5))

#         for i, (method, color) in enumerate(zip(methods, colors)):
#             method_df = bundle_df[bundle_df['method'] == method]
#             data = [method_df[method_df['num_patients'] == patients][bundle_column].values 
#                     for patients in bundle_df['num_patients'].unique()]
            
#             # Ensure there is data for each num_patients
#             if any(len(d) > 0 for d in data):
#                 positions = x + i * width  # Shift positions for each method
#                 ax.boxplot(data, positions=positions, widths=0.15, patch_artist=True, 
#                         boxprops=dict(facecolor=color, color=color),
#                         medianprops=dict(color='black'))
                
#         ax.set_xlabel('Nombre de patients')
#         ax.set_ylabel('Valeurs')
#         ax.set_title(f'Boxplots pour le bundle: {bundle_column} avec {prct * 100}% de malades')
#         ax.set_xticks(x + width * (len(methods) - 1) / 2)
#         ax.set_xticklabels(bundle_df['num_patients'].unique())
#         ax.legend(handles=[plt.Line2D([0], [0], color=color, lw=4, label=f'Method: {method}') for method, color in zip(methods, colors)])
#         plt.savefig(os.path.join(directory, f'{bundle_column}_boxplot.png'))
#         plt.close()

# # Exemple d'utilisation
# mea_df = pd.read_csv(os.path.join(MAINFOLDER, robust_method, "mea_compilation.csv"))
# add_nb_patients_and_diseased(mea_df)
# disease_ratios = [0.1, 0.3, 0.5, 0.7]
# for disease_ratio in disease_ratios:
#     plot_bundle(mea_df, disease_ratio, os.path.join(MAINFOLDER, robust_method, ANALYSISFOLDER))

## TESTS

In [26]:
# # # TEST ADD BIAIS
# # Split the data into training and testing sets
# directory = os.path.join(MAINFOLDER, "testBiais")
# os.makedirs(directory, exist_ok=True)
# train_df, test_df = split_train_test(CAMCAN, test_size=0.2, random_state=42)

# # Generate biased data
# # Save the original non-biased data to temporary files
# temp_train_file_original = os.path.join(directory, "temp_train_original.csv")
# temp_test_file_original = os.path.join(directory, "temp_test_original.csv")
# train_df.to_csv(temp_train_file_original, index=False)
# test_df.to_csv(temp_test_file_original, index=False)

# # Generate biased data
# sampled_df_biaied, test_df_biaised, gammas,deltas, ruffles= generate_biaised_data(train_df, test_df)

# # Save the biased data to temporary files
# temp_train_file = os.path.join(directory, "temp_train_biased.csv")
# temp_test_file = os.path.join(directory, "temp_test_biased.csv")
# sampled_df_biaied.to_csv(temp_train_file, index=False)
# test_df_biaised.to_csv(temp_test_file, index=False)

# # Run the combat_visualize_data script
# outname_train = os.path.join("visualize_train")
# cmd = (
#     "scripts/combat_visualize_data.py"
#     + " "
#     + temp_train_file_original
#     + " "
#     + temp_train_file
#     + " --out_dir "
#     + directory
#     + " --outname "
#     + outname_train
#     + " -f"
#     + " --bundles all"
# )
# subprocess.call(cmd, shell=True)

# # Display gammas and deltas along with their mean and standard deviation
# print("Gammas:", gammas)
# print("Deltas:", deltas)
# gammas = list(gammas.values())
# deltas = list(deltas.values())
# print("\nGamma Statistics:")
# print(f"Mean: {np.mean(gammas)}, Std: {np.std(gammas)}")

# print("\nDelta Statistics:")
# print(f"Mean: {np.mean(deltas)}, Std: {np.std(deltas)}")
# print("Ruffles:", ruffles)


In [27]:
# # TEST Powerpoint generation
# d  = os.path.join(MAINFOLDER, robust_method, "adni_100_Philips_3T")
# create_presentation(d)

In [28]:
# # TEST the sample_patients function with compilation data data
# sampled_df = sample_patients(COMPILATION, num_patients=100, disease_ratio=0.5)
# print(sampled_df)


In [29]:
# load_metrics("ROBUST/IQR/50_30/0/", "50_patients_30_percent_0")

In [30]:
# # Load the dists_compilation and metrics_compilation CSV files
# dists_compilation_path = os.path.join(directory, "dists_compilation.csv")
# metrics_compilation_path = os.path.join(directory, "metrics_compilation.csv")

# dists_compilation = pd.read_csv(dists_compilation_path)
# metrics_compilation = pd.read_csv(metrics_compilation_path)

# # Change the site column
# dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
# metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# # Display the means by site
# dists_means_by_site = dists_compilation.groupby(['site','comparaison']).mean()
# metrics_means_by_site = metrics_compilation.groupby('site').mean()

# print(dists_means_by_site)
# print(metrics_means_by_site)

In [31]:
# # FIX METRICS COMPILATION
# directory = os.path.join(MAINFOLDER, robust_method)
# df = pd.read_csv(os.path.join(directory, "metrics_compilation.csv"))

# # Group by the site
# grouped = df.groupby('site')

# # Process each site
# cleaned_dfs = []
# for site, group in grouped:
#     # Reset index for easier manipulation
#     group = group.reset_index(drop=True)
    
#     # # The first row is the "bundle row" (new column names)
#     # new_columns = group.iloc[0].values  # Extract column names from the first row
#     # new_columns[-1] = 'site'
#     # group = group.iloc[1:]  # Remove the first row
    
#     # # Assign new column names
#     # group.columns = new_columns
    
#     # # Sort the columns alphabetically (excluding 'site')
#     # sorted = group.sort_index(axis=1)
#     # Add a new column 'nomm' with the value indicating the metric for each row
#     metrics = ['tp', 'fp', 'tn', 'fn', 'precision', 'recall', 'taux_faux_positifs', 'f1_score']
#     group['metric'] = metrics
    
#     # # Append the cleaned DataFrame for this site
#     cleaned_dfs.append(group)

# # Concatenate all cleaned DataFrames
# final_df = pd.concat(cleaned_dfs, ignore_index=True)

# # Save or display the result
# final_df.to_csv(os.path.join(directory, "metrics_compilation.csv"), index=False)


In [32]:
# REAL SITES
# directory = os.path.join(MAINFOLDER, robust_method)
# raw_directory = os.path.join(RAWFOLDER, site_group)
# for filename in sorted(os.listdir(raw_directory)):
#     f = os.path.join(raw_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         analyse_site(f, robust_method, directory)
        
